In [3]:
!unzip -o "/content/Data Project.zip"

Archive:  /content/Data Project.zip
   creating: Data Project/
   creating: Data Project/1. Cơm trắng - 10.000/
  inflating: Data Project/1. Cơm trắng - 10.000/img (1).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (10).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (100).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (101).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (102).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (103).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (104).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (105).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (106).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (107).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (108).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (109).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (11).jpg  
  inflating: Data Project/1. Cơm trắng - 10.000/img (110).jp

In [4]:
# ================== TRAIN CNN TỪ ĐẦU CHO 17 LỚP ==================
import os, json
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 0) CẤU HÌNH CƠ BẢN
data_dir   = '/content/Data Project'   # Thư mục gốc, bên trong có 17 thư mục con (mỗi thư mục = 1 món)
IMG_SIZE   = (128, 128)
BATCH_SIZE = 32
VAL_SPLIT  = 0.2
SEED       = 42

# 1) CHUẨN BỊ DỮ LIỆU VỚI IMAGEDATAGENERATOR (KHÔNG DÙNG PRE-TRAIN)
#    Tách train/val bằng validation_split, giữ seed để tách ổn định.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT,
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.20,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VAL_SPLIT
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)

# In ra các lớp đã tìm thấy để kiểm tra
print("\nCác lớp mô hình sẽ học (tên_lớp: index):")
print(train_generator.class_indices)
num_classes = len(train_generator.class_indices)

# Lưu mapping lớp → index để dùng khi suy luận sau này
with open('classes.json', 'w') as f:
    json.dump(train_generator.class_indices, f, ensure_ascii=False, indent=2)
print(f"✅ Đã lưu nhãn vào classes.json (số lớp: {num_classes})")

# 2) XÂY DỰNG MÔ HÌNH (CNN từ đầu, KHÔNG pre-train)
#    Giữ đúng “cấu trúc thầy dạy”: các lớp Conv2D → MaxPooling2D → Flatten → Dense → Dropout → Dense.
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Thêm 1 block conv nhẹ cho đủ sức phân biệt 17 lớp (vẫn là CNN scratch)
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Tự động khớp số lớp theo thư mục
])

# 3) COMPILE MÔ HÌNH
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# 4) HUẤN LUYỆN MÔ HÌNH
#    Nếu dữ liệu mỗi lớp ít, 25–40 epochs là hợp lý; để 50 kèm early-stop là tốt nhất.
#    Để giữ đúng “mẫu thầy”, mình không thêm callbacks bắt buộc. Bạn có thể bật nhanh ở dưới nếu muốn.
EPOCHS = 30

# (TÙY CHỌN) CALLBACKS: bật nếu muốn dừng sớm & lấy best model — vẫn không ảnh hưởng cấu trúc chính
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=4, min_lr=1e-6, verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
]

print("\nBắt đầu huấn luyện...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=val_generator,
    callbacks=callbacks  # muốn “y như mẫu thầy” thì bỏ tham số này đi
)

# 5) LƯU LẠI THÀNH QUẢ
model.save('food17_scratch.h5')   # đặt tên rõ ràng: CNN scratch cho 17 lớp
print("\n✅ Huấn luyện hoàn tất! Mô hình đã được lưu vào file 'food17_scratch.h5'")
print("💾 Ngoài ra có 'best_model.h5' (nếu callbacks đã bật) và 'classes.json' (mapping nhãn).")


Found 1759 images belonging to 17 classes.
Found 432 images belonging to 17 classes.

Các lớp mô hình sẽ học (tên_lớp: index):
{'1. Cơm trắng - 10.000': 0, '10. Rau xào - 10000': 1, '11. Trứng chiên - 25000': 2, '12. Canh bí đao - 12000': 3, '13. Canh bí đỏ - 12000': 4, '14. Dưa leo - 5000': 5, '15. Lạp sưởng - 15000': 6, '16. Nước chấm - 3000': 7, '17. Khay trống - 0': 8, '2. Đậu hũ sốt cà - 25000': 9, '3. Cá hú kho - 30000': 10, '4. Thịt kho trứng - 30000': 11, '5. Thịt kho - 25000': 12, '6. Canh chua cá - 25000': 13, '7. Canh chua - 10000': 14, '8. Sường nướng - 30000': 15, '9. Canh rau - 7000': 16}
✅ Đã lưu nhãn vào classes.json (số lớp: 17)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 17)             │         4,369 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,520,401 (24.87 MB)

...

55/55 ━━━━━━━━━━━━━━━━━━━━ 31s 565ms/step - accuracy: 0.8401 - loss: 0.4753 - val_accuracy: 0.9190 - val_loss: 0.3477 - learning_rate: 0.0010
Epoch 14/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.8675 - loss: 0.3862
Epoch 14: val_accuracy did not improve from 0.91898
55/55 ━━━━━━━━━━━━━━━━━━━━ 30s 553ms/step - accuracy: 0.8675 - loss: 0.3858 - val_accuracy: 0.8958 - val_loss: 0.3883 - learning_rate: 0.0010
Epoch 15/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.8498 - loss: 0.3986
Epoch 15: val_accuracy did not improve from 0.91898
55/55 ━━━━━━━━━━━━━━━━━━━━ 31s 554ms/step - accuracy: 0.8499 - loss: 0.3984 - val_accuracy: 0.9144 - val_loss: 0.2592 - learning_rate: 0.0010
Epoch 16/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.9002 - loss: 0.2866
Epoch 16: val_accuracy did not improve from 0.91898
55/55 ━━━━━━━━━━━━━━━━━━━━ 30s 544ms/step - accuracy: 0.9002 - loss: 0.2867 - val_accuracy: 0.9074 - val_loss: 0.3290 - learning_rate: 0.0010
Epoch 17/30
55/

55/55 ━━━━━━━━━━━━━━━━━━━━ 28s 517ms/step - accuracy: 0.9161 - loss: 0.2270 - val_accuracy: 0.9444 - val_loss: 0.2806 - learning_rate: 5.0000e-04
Epoch 21/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 434ms/step - accuracy: 0.9336 - loss: 0.1933
Epoch 21: val_accuracy did not improve from 0.94444
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 523ms/step - accuracy: 0.9335 - loss: 0.1934 - val_accuracy: 0.9306 - val_loss: 0.1863 - learning_rate: 5.0000e-04
Epoch 22/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step - accuracy: 0.9340 - loss: 0.1998
Epoch 22: val_accuracy did not improve from 0.94444
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 519ms/step - accuracy: 0.9340 - loss: 0.1996 - val_accuracy: 0.9375 - val_loss: 0.1930 - learning_rate: 5.0000e-04
Epoch 23/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step - accuracy: 0.9356 - loss: 0.1680
Epoch 23: val_accuracy did not improve from 0.94444
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 522ms/step - accuracy: 0.9357 - loss: 0.1680 - val_accuracy: 0.9282 - val_loss: 0.2771 - learning_rate: 5.0000e-04

55/55 ━━━━━━━━━━━━━━━━━━━━ 30s 549ms/step - accuracy: 0.9668 - loss: 0.0999 - val_accuracy: 0.9560 - val_loss: 0.1618 - learning_rate: 2.5000e-04
Epoch 28/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step - accuracy: 0.9668 - loss: 0.1079
Epoch 28: val_accuracy did not improve from 0.95602
55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 523ms/step - accuracy: 0.9668 - loss: 0.1079 - val_accuracy: 0.9491 - val_loss: 0.2200 - learning_rate: 2.5000e-04
Epoch 29/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step - accuracy: 0.9645 - loss: 0.0990
Epoch 29: val_accuracy did not improve from 0.95602
55/55 ━━━━━━━━━━━━━━━━━━━━ 30s 540ms/step - accuracy: 0.9644 - loss: 0.0992 - val_accuracy: 0.9444 - val_loss: 0.1650 - learning_rate: 2.5000e-04
Epoch 30/30
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 452ms/step - accuracy: 0.9723 - loss: 0.0779
Epoch 30: val_accuracy improved from 0.95602 to 0.95833, saving model to best_model.h5


55/55 ━━━━━━━━━━━━━━━━━━━━ 29s 536ms/step - accuracy: 0.9723 - loss: 0.0782 - val_accuracy: 0.9583 - val_loss: 0.1323 - learning_rate: 2.5000e-04



✅ Huấn luyện hoàn tất! Mô hình đã được lưu vào file 'food17_scratch.h5'
💾 Ngoài ra có 'best_model.h5' (nếu callbacks đã bật) và 'classes.json' (mapping nhãn).
